In [15]:
import json
from pathlib import Path
import re
import random
import pandas as pd
import re

In [16]:
import logging
import pandas as pd
import numpy as np
from numpy import random
import gensim
import nltk
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import re
from bs4 import BeautifulSoup
%matplotlib inline

 # Data Preprocessing

In [406]:
data=pd.read_csv("Amazon_reviews_6T.csv")

In [387]:


REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
    text = BeautifulSoup(text, "lxml").text # HTML decoding
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
data['body'] = data['body'].apply(clean_text)


In [388]:
X = data[data['Class'].notnull()].body
y = data[data['Class'].notnull()].Class
#y = y.map({'Fingerprint': 1, 'Value for money': 2,'Sound Quality':3,'Battery life':4,'Not good':5,'Picture Quality':6})

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state = 0)

#  Classification Model 

## Naive Bayes

In [281]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer

nb = Pipeline([('vect', CountVectorizer()),
               ('tfidf', TfidfTransformer()),
               ('clf', MultinomialNB()),
              ])
nb.fit(X_train, y_train)


from sklearn.metrics import classification_report
y_pred = nb.predict(X_test)




In [283]:
print(classification_report(y_test, y_pred))
print('accuracy %s' % accuracy_score(y_pred, y_test))

C:\Users\sushbiswal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


                 precision    recall  f1-score   support

   Battery life       0.00      0.00      0.00         2
    Fingerprint       0.00      0.00      0.00         2
Picture Quality       0.00      0.00      0.00         1
  Sound Quality       0.00      0.00      0.00         2
Value for money       0.56      1.00      0.72         9

      micro avg       0.56      0.56      0.56        16
      macro avg       0.11      0.20      0.14        16
   weighted avg       0.32      0.56      0.40        16



## SGD Classifier

In [285]:
from sklearn.linear_model import SGDClassifier

sgd = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', SGDClassifier(loss='hinge', penalty='l2',alpha=1e-3, random_state=42, max_iter=5, tol=None)),
               ])
sgd.fit(X_train, y_train)



y_pred = sgd.predict(X_test)

print(classification_report(y_test, y_pred))
print('accuracy %s' % accuracy_score(y_pred, y_test))

C:\Users\sushbiswal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\stochastic_gradient.py:183: FutureWarning: max_iter and tol parameters have been added in SGDClassifier in 0.19. If max_iter is set but tol is left unset, the default value for tol in 0.19 and 0.20 will be None (which is equivalent to -infinity, so it has no effect) but will change in 0.21 to 1e-3. Specify tol to silence this warning.
  FutureWarning)


                 precision    recall  f1-score   support

   Battery life       0.00      0.00      0.00         2
    Fingerprint       0.67      1.00      0.80         2
Picture Quality       1.00      1.00      1.00         1
  Sound Quality       0.00      0.00      0.00         2
Value for money       0.67      0.89      0.76         9

      micro avg       0.69      0.69      0.69        16
      macro avg       0.47      0.58      0.51        16
   weighted avg       0.52      0.69      0.59        16

accuracy 0.6875


## Logistic Regression 

In [328]:
from sklearn.linear_model import LogisticRegression

logreg = Pipeline([('vect', CountVectorizer()),
                ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(n_jobs=1, C=1e5)),
               ])
logreg.fit(X_train, y_train)



y_pred = logreg.predict(X_test)



In [329]:
print(classification_report(y_test, y_pred))
print('accuracy %s' % accuracy_score(y_pred, y_test))

                 precision    recall  f1-score   support

   Battery life       0.00      0.00      0.00         2
    Fingerprint       0.33      0.50      0.40         2
Picture Quality       1.00      1.00      1.00         1
  Sound Quality       0.00      0.00      0.00         2
Value for money       0.64      0.78      0.70         9

      micro avg       0.56      0.56      0.56        16
      macro avg       0.39      0.46      0.42        16
   weighted avg       0.46      0.56      0.51        16

accuracy 0.5625


In [400]:
v=0
for i in data[data['Class'].isnull()].index:
    print(i)
     
    data.at[i, 'Class'] = logreg.predict(data[data['Class'].isnull()].body)[v]
v=v+1

43
44
45
46
47
48
49
50
51
52
53
54
56
57
58
59
60
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
120
121
122
123
124
125
126
127
128
129


In [ ]:
data.to_csv("Final_predictedClasses.csv")

# Doc2Vec implementation


In [396]:
from tqdm import tqdm
tqdm.pandas(desc="progress-bar")
from gensim.models import Doc2Vec
from sklearn import utils
import gensim
from gensim.models.doc2vec import TaggedDocument
import re

In [76]:
import gensim
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [267]:
def label_sentences(corpus, label_type):
    """
    Gensim's Doc2Vec implementation requires each document/paragraph to have a label associated with it.
    We do this by using the TaggedDocument method. The format will be "TRAIN_i" or "TEST_i" where "i" is
    a dummy index of the post.
    """
    labeled = []
    for i, v in enumerate(corpus):
        label = label_type + '_' + str(i)
        labeled.append(TaggedDocument(v.split(), [label]))
    return labeled


In [268]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0, test_size=0.3)
X_train = label_sentences(X_train, 'Train')
X_test = label_sentences(X_test, 'Test')
all_data = X_train + X_test

In [269]:
model_dbow = Doc2Vec(dm=0, vector_size=300, negative=5, min_count=1, alpha=0.065, min_alpha=0.065)
model_dbow.build_vocab([x for x in tqdm(all_data)])

for epoch in range(30):
    model_dbow.train(utils.shuffle([x for x in tqdm(all_data)]), total_examples=len(all_data), epochs=1)
    model_dbow.alpha -= 0.002
    model_dbow.min_alpha = model_dbow.alpha

100%|███████████████████████████████████████████████████████████████████████████████| 46/46 [00:00<00:00, 22941.50it/s]


In [270]:
def get_vectors(model, corpus_size, vectors_size, vectors_type):
    """
    Get vectors from trained doc2vec model
    :param doc2vec_model: Trained Doc2Vec model
    :param corpus_size: Size of the data
    :param vectors_size: Size of the embedding vectors
    :param vectors_type: Training or Testing vectors
    :return: list of vectors
    """
    vectors = np.zeros((corpus_size, vectors_size))
    for i in range(0, corpus_size):
        prefix = vectors_type + '_' + str(i)
        vectors[i] = model.docvecs[prefix]
    return vectors
    
train_vectors_dbow = get_vectors(model_dbow, len(X_train), 300, 'Train')
test_vectors_dbow = get_vectors(model_dbow, len(X_test), 300, 'Test')

In [271]:
logreg = LogisticRegression(n_jobs=1, C=1e5)
logreg.fit(train_vectors_dbow, y_train)
logreg = logreg.fit(train_vectors_dbow, y_train)
y_pred = logreg.predict(test_vectors_dbow)



C:\Users\sushbiswal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
C:\Users\sushbiswal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\sushbiswal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


In [272]:
print(classification_report(y_test, y_pred))
print('accuracy %s' % accuracy_score(y_pred, y_test))

C:\Users\sushbiswal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Users\sushbiswal\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


              precision    recall  f1-score   support

           1       0.00      0.00      0.00         2
           2       0.50      0.50      0.50         8
           3       0.00      0.00      0.00         2
           4       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         0
           6       0.00      0.00      0.00         1

   micro avg       0.29      0.29      0.29        14
   macro avg       0.08      0.08      0.08        14
weighted avg       0.29      0.29      0.29        14

accuracy 0.2857142857142857
